# CART Implementation

### Import Packages

In [1]:
import math
import pandas as pd
import numpy as np
import requests
from io import StringIO
from gurobipy import *
from sklearn import tree
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from ucimlrepo import fetch_ucirepo as fetc
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler



### Import Data and Check Data Status

In [13]:
rice = fetc(id=545)
X = rice.data.features 
y = rice.data.targets
df = pd.DataFrame(X, columns=rice.data.feature_names).sample(n=3000, random_state=42)
df['target'] = y
df = df.reset_index(drop=True)

#Change non numeric columns to numeric
column_names = ["Area", "Perimeter", "Major_Axis_Length", "Minor_Axis_Length","Eccentricity", "Convex_Area", "Extent", "target"]
non_numeric_cols = df.drop(['target'],axis=1).select_dtypes(exclude=[np.number]).columns
if not non_numeric_cols.empty:
    for col in non_numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
for col in column_names[:len(column_names)-1]:
    df[col] = StandardScaler().fit_transform(df[[col]])
    
for col in column_names[:len(column_names)-1]:
    df[col] = MinMaxScaler().fit_transform(df[[col]])

#Convert target column to numeric
le = LabelEncoder()
df['target'] = le.fit_transform(df['target'])
        
df.info()
#df.isnull().sum()
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3000 non-null   float64
 1   Perimeter          3000 non-null   float64
 2   Major_Axis_Length  3000 non-null   float64
 3   Minor_Axis_Length  3000 non-null   float64
 4   Eccentricity       3000 non-null   float64
 5   Convex_Area        3000 non-null   float64
 6   Extent             3000 non-null   float64
 7   target             3000 non-null   int64  
dtypes: float64(7), int64(1)
memory usage: 187.6 KB
       Area  Perimeter  Major_Axis_Length  Minor_Axis_Length  Eccentricity  \
0  0.430470   0.530431           0.453409           0.576023      0.644061   
1  0.427478   0.411490           0.370038           0.610231      0.548847   
2  0.467875   0.475215           0.391069           0.662542      0.519751   
3  0.487502   0.601254           0.615399         

### Split Data into Training and Testing Sets

In [14]:
X = df.drop(['target'], axis=1)
y = df['target']
print("Total number of Class: " + str(len(np.unique(y))) + " which are: " + str(np.unique(y)))
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

Total number of Class: 2 which are: [0 1]


### Decision Tree

In [15]:
def clf(x_train, y_train, x_test, y_test,K):
    clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=K, random_state=42)
    clf_gini.fit(x_train, y_train)
    y_pred_gini = clf_gini.predict(x_test)
    y_pred_gini_train = clf_gini.predict(x_train)
    print('Testing Set Accuracy Score: ', accuracy_score(y_test, y_pred_gini), 'Training Set Accuracy Score:',accuracy_score(y_train, y_pred_gini_train))

clf(x_train, y_train, x_test, y_test,3)

Testing Set Accuracy Score:  0.9111111111111111 Training Set Accuracy Score: 0.9341176470588235


### Tree Structure: Will be Used for Warmstart of OCT

In [16]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
clf.fit(x_train, y_train)

"""
The following is for warm start in OCT
"""
feature = clf.tree_.feature
threshold = clf.tree_.threshold
feature_indices = []
threshold_indices = []
feature_indices_leaf =[]
threshold_indices_leaf = []
for i in range(len(feature)):
    if feature[i] != -2 and feature[i+1] != -2:
        feature_indices.append(int(feature[i]))
        threshold_indices.append(float(threshold[i]))
    elif feature[i] != -2 and feature[i+1] == -2:
        feature_indices_leaf.append(int(feature[i]))
        threshold_indices_leaf.append(float(threshold[i]))
    else:
        continue
feature_indices.extend(feature_indices_leaf)
threshold_indices.extend(threshold_indices_leaf)
print("Feature Indices: ", feature_indices, "Threshold Indices:", threshold_indices)
#leaf nodes
#branch nodes
print(np.sum(clf.tree_.feature == -2))
internal_nodes = np.sum(clf.tree_.feature != -2)
total_nodes = clf.tree_.node_count
print(f"Internal nodes: {internal_nodes}, Total nodes: {total_nodes}")
        

Feature Indices:  [2, 2, 2, 2, 2, 5, 2] Threshold Indices: [0.5125517845153809, 0.44264456629753113, 0.5738876163959503, 0.38609644770622253, 0.47788456082344055, 0.5260196924209595, 0.6545738279819489]
8
Internal nodes: 7, Total nodes: 15


In [17]:
total_nodes = clf.tree_.node_count
leaf_nodes = round(total_nodes / 2)
branch_nodes = total_nodes // 2
print(f"Leaf nodes: {leaf_nodes}, Branch nodes: {branch_nodes}")

Leaf nodes: 8, Branch nodes: 7


# OCT Implementation

### Import Packages

In [18]:
import gurobipy as gp
from gurobipy import GRB

In [19]:
m = gp.Model('OCT')

### Construct Constraints

In [54]:
"""
training data (X,Y)=(x_i,y_i), i=1,...,n; x_i in R^p and normalized to [0,1]^p, y_i in {1,...,K}
-n = # obervations
-p = # features
-K = # class label

-p(t) = parent nodes of node t
-A_L(t) = {left_branch ancestors of t} = {t if tmod2=0 and t/2 recursively gives the set}
-A_R(t) = {right_branch ancestors of t} = {t if (t-1)mod2=0 and (t-1)/2 recursively gives the set}


-D = max depth
-T = 2^(D+1)-1 = max # nodes
-TB = branch nodes = left nodes = {1,...,floor(T/2)} applies split is ax<b 
-TL = leaf = {floor(T/2)+1,...,T} make class prediction
-?a_t in R^p
-?b_t in R
-p = # features

-d_t = 1{node t applies a split}
-sum a_jt = d_t, j=1,...p, t in TB
-0 <= b_t <= d_t, t in TB
-a_jt in {0,1}, j=1,...p, t in TB


-d_t <= d_p(t), t in TB/{1}

-z_it = 1{x_i in node t}
-l_t = 1{leaf t contains any point}
-z_it <= l_t, t in TB
-sum(z_it) >= N_min*l_t for i=1,...,n, t in TB
-sum(z_it)=1 for t in TB, i=1,...,n

-x_j^i = ith largest value in feature j
-epsilon_j = min{x_j^(i+1)-x_j^i, i=1,...,n}
-epsilon_max = max{epsilon_j} wrt j
-epsilon  = {epsilon_1,...,epsilon_p}
-a_m(x_t + epsilon) <= b_m +(1+epsilon_max)(1-z_it), i=1,...,n, for all t in TB, for all m in A_L(t)
-a_m*x_i >= b_m - (1-z_it), i=1,...,n, for all t in TB, for all m in A_R(t)

-Y_ik = {1 if y_i=k, -1 otherwise}, k=1,...,K, i=1,...n
N_kt = 0.5*sum((1+Y_ik)z_it) for i=1,...,n; k=1,...K, t in TL
N_t = sum(z_it) for i=1,...,n; t in TL
c_t = argmax{N_kt} wrt k
c_kt = 1{c_t = k}
sum(c_kt) = l_t wrt k

L_t = N_t - max{N_kt} wrt k = min{N_t - N_kt} wrt k
L_t >= N_t - N_kt - n(1-c_kt), k=1,...K, t in TL
L_t <= N_t - N_kt + n*c_kt, k=1,...K, t in TL
L_t >= 0

L^ = baseline accuracy = #{most popular class}/n

objective: min (1/L^)sum(L_t) for t in TL + alpha*sum(d_t) for t in TB

"""

'\ntraining data (X,Y)=(x_i,y_i), i=1,...,n; x_i in R^p and normalized to [0,1]^p, y_i in {1,...,K}\n-n = # obervations\n-p = # features\n-K = # class label\n\n-p(t) = parent nodes of node t\n-A_L(t) = {left_branch ancestors of t} = {t if tmod2=0 and t/2 recursively gives the set}\n-A_R(t) = {right_branch ancestors of t} = {t if (t-1)mod2=0 and (t-1)/2 recursively gives the set}\n\n\n-D = max depth\n-T = 2^(D+1)-1 = max # nodes\n-TB = branch nodes = left nodes = {1,...,floor(T/2)} applies split is ax<b \n-TL = leaf = {floor(T/2)+1,...,T} make class prediction\n-?a_t in R^p\n-?b_t in R\n-p = # features\n\n-d_t = 1{node t applies a split}\n-sum a_jt = d_t, j=1,...p, t in TB\n-0 <= b_t <= d_t, t in TB\n-a_jt in {0,1}, j=1,...p, t in TB\n\n\n-d_t <= d_p(t), t in TB/{1}\n\n-z_it = 1{x_i in node t}\n-l_t = 1{leaf t contains any point}\n-z_it <= l_t, t in TB\n-sum(z_it) >= N_min*l_t for i=1,...,n, t in TB\n-sum(z_it)=1 for t in TB, i=1,...,n\n\n-x_j^i = ith largest value in feature j\n-epsilo

### Predetermined Variables

In [ ]:
n= df.shape[0] # number of observations
p = df.drop(['target'],axis=1).shape[1] # number of features
K = len(np.unique(df['target'])) # number of classes
D = 3 # max depth
T = 2**(D+1)-1 # max number of nodes
L_hat = df['target'].value_counts().max()/n # baseline accuracy, predicting the most popular class for the dataset

N_min = math.floor(n*0.05)

#Epsilon
epsilon=[]
for j in range(p):
    x_j = df.iloc[:,j].tolist()
    x_j.sort()  
    e=[]
    for i in range(n-1):
        if x_j[i+1]!=x_j[i]:
            e.append(x_j[i+1] - x_j[i])
    epsilon.append(min(e))
epsilon_max = max(epsilon)

print("epsilon: " + str(epsilon))
print("epsilon_max: " + str(epsilon_max))

#Y Matrix
Y = np.zeros([n,K], dtype = int) - 1 # Y_ik
Y[df.index,  y] = 1  #based on the sample code, the [x,y] x is the features index, y is the class index



epsilon: [8.801267382496647e-05, 5.157226001362147e-06, 3.273563998762441e-07, 1.5892507770898234e-07, 3.5751314475529483e-07, 8.790436005612356e-05, 1.6530101976108824e-07]
epsilon_max: 8.801267382496647e-05


### Predetermined Sets (Note that index starts from 1)

In [21]:
parent = []
for t in range(1,T+1):
    if t%2==0:
        parent.append(t//2)
    else:
        parent.append((t-1)//2)
print(parent)

left_ancestors = []
right_ancestors = []
for t in range(1,T+1):
    la_t =[]
    ra_t =[]
    tau=t
    while tau>1:
        pt = tau//2
        if tau % 2 == 0: #if t is even, then its parent is a left ancestor, else is a right ancestor
            la_t.append(pt)
        else:
            ra_t.append(pt)
        tau = pt
    la_t.sort() 
    ra_t.sort()
    left_ancestors.append(la_t)
    right_ancestors.append(ra_t)

TB = list(range(1,math.floor((T+1)/2)))  #Branch nodes
TL = list(range(math.floor((T+1)/2),T+1)) #Leaf nodes
TT = list(range(1,T+1)) #total nodes

print("Branch nodes: ", TB)
print("Leaf nodes: ", TL)

print("left_ancestors: " + str(left_ancestors))
print("right_ancestors: " + str(right_ancestors))

[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7]
Branch nodes:  [1, 2, 3, 4, 5, 6, 7]
Leaf nodes:  [8, 9, 10, 11, 12, 13, 14, 15]
left_ancestors: [[], [1], [], [1, 2], [1], [3], [], [1, 2, 4], [1, 2], [1, 5], [1], [3, 6], [3], [7], []]
right_ancestors: [[], [], [1], [], [2], [1], [1, 3], [], [4], [2], [2, 5], [1], [1, 6], [1, 3], [1, 3, 7]]


### Variables

In [22]:
a = m.addVars(p,TB, vtype=GRB.BINARY, name="a_t") #dim |TB|xp
b = m.addVars(TB, vtype=GRB.CONTINUOUS, lb = 0, ub = 1, name="b_t") #dim |TB|
d = m.addVars(TB, vtype=GRB.BINARY, name="d_t") #dim |TB|
z = m.addVars(n, TL, vtype=GRB.BINARY, name="z") #dim nx|TL|
l = m.addVars(TL, vtype=GRB.BINARY, name="l_t") #dim |TL|
Nk = m.addVars(K, TL, vtype=GRB.INTEGER,name="N_kt") #dim Kx|TL|
N = m.addVars(TL, vtype=GRB.INTEGER, name="N_t") #dim |TL|
ck = m.addVars(K, TL, vtype=GRB.BINARY, name="c_kt")
L = m.addVars(TL, name="L_t") 

### Warm Start

In [23]:
# warm start using the results of CART algorithm
for i in TB:
    a[feature_indices[i-1], i].start = 1
    b[i].start = threshold_indices[i-1]

### Constraints

In [24]:
for t in TB:
    m.addConstr(a.sum("*",t) == d[t], name="sum_constraint_of_ajt") # sum of ajt = dt
    m.addConstr(b[t] <= d[t], name="bt_constraint_dt")     # bt <= dt
    m.addConstr(d[t] == 1, name="dt_constraint_d(t)")  #force dt = 1???

for t in TB[1:]:
    m.addConstr(d[t] <= d[t//2], name="dt_constraint_dp(t)") # dt <= dp(t)    

for t in TL:
    m.addConstr(z.sum("*",t) >= N_min*l[t], name="sum_of_zt_constraint_Nmin_lt") # sum of zit >= Nmin*lt
    m.addConstr(z.sum(i,"*") == 1, name="sum_of_zi(t)_constraint_1")  # sum sum of zit = 1
    for i in range(n):
        m.addConstr(z[i, t] <= l[t]) # zit <= lt
    
    for k in range(K):
        m.addConstr(Nk[k,t] == 1/2 * gp.quicksum(z[i,t] * (Y[i,k] + 1) for i in range(n))) #Nkt = 1/2(sum of (1+Yik)*zit #may need to be corrected

    m.addConstr(N[t] == z.sum("*",t))  # Nt = sum of zit

    m.addConstr(l[t] == ck.sum("*",t)) # sum of ckt = lt
    
    m.addConstr(l[t] == 1, name="dt_constraint_l(t)")
    
for t in TL:
    l_ancestors = left_ancestors[t - 1]  # cache the list
    if l_ancestors:
        for la in l_ancestors:
            for i in range(n):
                xi = X.iloc[i]  # cache row once
                m.addConstr(gp.quicksum(a[j, la] * (xi[j] + epsilon[j]) for j in range(p)) <= b[la] + (1 + epsilon_max) * (1 - z[i, t]),
                    name=f"split_l_{la}_{i}_{t}"
                )
                
    r_ancestors = right_ancestors[t - 1]
    if r_ancestors:
        for r in r_ancestors:
            for i in range(n):
                xi = X.iloc[i]  # cache the row once
                m.addConstr(
                    gp.quicksum(a[j, r] * xi[j] for j in range(p)) >= b[r] - (1 - z[i, t]),
                    name=f"split_r_{r}_{i}_{t}"
                )
        
for t in TL:
    m.addConstr(L[t] >= 0, name="Lt_constraint3") #Lt ≥ 0
    for k in range(K):
        m.addConstr(L[t] >= N[t] - Nk[k,t] - n*(1-ck[k,t]), name="Lt_constraint1") #Lt ≤ Nt − Nkt + n(1-ckt) 
        m.addConstr(L[t] <= N[t] - Nk[k,t] + n*ck[k,t], name="Lt_constraint2")  #Lt ≤ Nt − Nkt + n*ckt 


In [25]:
m.update()
m.setObjective(L.sum('*') / L_hat + 0.5*gp.quicksum(d[t] for t in TB), GRB.MINIMIZE) #minimize L + alpha*sum(d_t) for t in TB

### Objective OCT

In [26]:
m.Params.timelimit = 600
m.optimize()

Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Non-default parameters:
TimeLimit  600

Optimize a model with 96123 rows, 24119 columns and 768269 nonzeros
Model fingerprint: 0x694541ec
Variable types: 15 continuous, 24104 integer (24080 binary)
Coefficient statistics:
  Matrix range     [2e-07, 3e+03]
  Objective range  [5e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]

User MIP start did not produce a new incumbent solution

Presolve removed 24066 rows and 31 columns
Presolve time: 0.99s
Presolved: 72057 rows, 24088 columns, 672117 nonzeros
Variable types: 7 continuous, 24081 integer (24057 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...



In [28]:
for i in TB:
    tmp4 = m.getVarByName('d' + '[' + str(i) + ']')
print(tmp4)

None


### Obtain the Tree Structure (OCT)

In [ ]:
"""
Tree Structure: 

Variables a, b, z, N_k and c=argmax{N_kt} are the decisive variables for the tree structure.

Take a point x_i, if <a_1,x_i> <= b_1, then x_i is down to node 2, otherwise down to node 3;
suppose x_i gets to node 2, then if <a_2,x_i> <= b_2, then x_i will be in node 4, otherwise node 5.

Such process continues until x_i reaches a leaf node.

For each leaf node t, we then count the number of points in each class k, which is N_kt.

max{N_kt} represents the most popular class, which is c_t, in leaf node t, but not every point in 
such leaf is of class c_t.

If L_t = 0, then every point in leaf node t is indeed of class c_t, and the tree is pure. 

So L_t is the number of misclassified points in leaf node t.

Since every point can only be in one of the leaf nodes, the sum of L_t is the total number of
misclassified points in the tree.

For a testing set, we create a tree with the splitting criterion based on a and b; then N_kt and L_t are what
need to be calculated to access the accuracy of the tree on the testing set. The number of splits in each level
of the testing tree is then number of observations times the number of branch nodes. 

The problem is then how to perform the splits efficiently? 
"""

In [37]:
a_matrix = np.zeros((p, len(TB)))
threshold_oct = np.zeros(len(TB))
split_position = np.zeros(len(TB))

for i in TB:
    split_position[i-1] = d[i].X

for i in TB:
    threshold_oct[i-1] = b[i].X
    
for i in range(p):
    for t_idx, t in enumerate(TB):  # convert TB to indexable order
        a_matrix[i, t_idx] = a[i, t].X

print(a_matrix, threshold_oct,split_position)

[[ 1. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0.]
 [-0. -0.  1. -0. -0. -0. -0.]
 [-0. -0. -0. -0.  1. -0. -0.]
 [-0. -0. -0. -0. -0. -0.  1.]
 [-0.  1. -0.  1. -0. -0. -0.]
 [-0. -0. -0. -0. -0.  1. -0.]] [0.42508449 0.33966245 0.58776865 0.30678622 0.56505417 0.45876304
 0.80144225] [1. 1. 1. 1. 1. 1. 1.]


In [22]:
# Obtain the coefficients of a and b
coff_a = np.zeros([p, len(TB)], dtype = int) 
coff_b = np.zeros(len(TB))

for i in TB:
    tmp1 = m.getVarByName('b_t[' + str(i) + ']')
    coff_b[i-1] = tmp1.X  #i-1 due to TB is [1, 2, 3, 4, 5, 6, 7] (make sure it is not out of range)

    for j in range(p):
        tmp2 = m.getVarByName('a_t[' + str(j) + ',' + str(i) + ']')
        coff_a[j, i-1] = int(tmp2.X)

# Obtain the labels of leaf nodes
labels = np.zeros(len(TL), dtype = int) - 1
coff_c = np.zeros([K, len(TL)], dtype = int)

for i in range(K):
    for j_idx, j in enumerate(TL): #Note TL is [8, 9, 10, 11, 12, 13, 14, 15] reset the index to [0,1,2,3,4,5,6,7]
        tmp3 = m.getVarByName('c_kt[' + str(i) + ',' + str(j) + ']')
        coff_c[i, j_idx] = int(tmp3.X)

k_idx, t_idx = np.where(coff_c == 1)

for i in range(len(k_idx)):
    labels[t_idx[i]] = k_idx[i]            



In [25]:
print("coff_a: " + str(coff_a))
print("coff_b: " + str(coff_b))
print("coff_c: " + str(coff_c))

coff_a: [[0 1 0 1 1 1 0]
 [0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
coff_b: [0.67287182 0.49712215 0.7343464  0.30153485 0.62587934 0.74078022
 0.61421259]
coff_c: [[0 0 0 1 1 1 1 1]
 [1 1 1 0 0 0 0 0]]


# OCT-H

Preliminary 

In [29]:
m_h = gp.Model('OCT-H')
mu = 0.005  #Base on the paper mu is a sufficiently small constant
alpha = 0.5
#M = 2 

Variables: Introduce s and a_hat as New Variables for OCT-H

In [30]:
a = m_h.addVars(p,TB, vtype=GRB.BINARY, name="a_t") #dim px|TB|
a_hat = m_h.addVars(p, TB, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="a_hat") #dim px|TB|
b = m_h.addVars(TB, vtype=GRB.CONTINUOUS, lb = 0, ub = 1, name="b_t") #dim |TB|
d = m_h.addVars(TB, vtype=GRB.BINARY, name="d_t") #dim |TB|
s = m_h.addVars(p,TB, vtype=GRB.BINARY, name="s") #dim px|TB|
z = m_h.addVars(n, TL, vtype=GRB.BINARY, name="z") #dim nx|TL|
l = m_h.addVars(TL, vtype=GRB.BINARY, name="l_t") #dim |TL|
Nk = m_h.addVars(K, TL, vtype=GRB.INTEGER,name="N_kt") #dim Kx|TL|
N = m_h.addVars(TL, vtype=GRB.INTEGER, name="N_t") #dim |TL|
ck = m_h.addVars(K, TL, vtype=GRB.BINARY, name="c_kt")
L = m_h.addVars(TL, name="L_t") 

Warmstart

In [31]:
# warm start using the results of CART algorithm
for i in TB:
    a[feature_indices[i-1], i].start = 1
    b[i].start = threshold_indices[i-1]

### Variables and Constraints (OCT-H)

In [32]:
for t in TB:
    m_h.addConstr(d[t] == 1, name="dt_constraint_d(t)") 
    m_h.addConstr(b[t] <= d[t], name="bt_constraint1_dt")     # bt <= dt
    m_h.addConstr(-d[t] <= b[t], name="bt_constraint2_dt")    #-dt <= bt
    
    m_h.addConstr(s.sum("*",t) >= d[t], name="sum_constraint_of_sjt")  #sum of sjt >= dt
    for i in range(p):
        m_h.addConstr(s[i,t] <= d[t], name="s_constraint1_dt")     # sjt <= dt
        m_h.addConstr(-s[i,t] <= a[i,t], name="a_constraint1_-st")  # -sjt <= ajt
        m_h.addConstr(a[i,t] <= s[i,t], name="a_constraint2_st")   # ajt <= sjt  

    m_h.addConstr(a_hat.sum("*",t) <= d[t], name="sum_constraint_of_a_hat") #sum of a_hat_jt >= dt
    for i in range(p):
        m_h.addConstr(a_hat[i,t] >= -a[i,t], name="a_hat_constraint_-a") #a_hat >= -ajt 
        m_h.addConstr(a_hat[i,t] >= a[i,t], name="a_hat_constraint_a")   #a_hat >= ajt  

for t in TB[1:]:
    m_h.addConstr(d[t] <= d[t//2], name="dt_constraint_dp(t)") # dt <= dp(t)   

for t in TL:
    m_h.addConstr(z.sum("*",t) >= N_min*l[t], name="sum_of_zt_constraint_Nmin_lt") # sum of zit >= Nmin*lt
    m_h.addConstr(z.sum(i,"*") == 1, name="sum_of_zi(t)_constraint_1")  # sum sum of zit = 1
    
    for i in range(n):
        m_h.addConstr(z[i, t] <= l[t]) # zit <= lt
  
    for k in range(K):
        m_h.addConstr(Nk[k,t] == 1/2 * gp.quicksum(z[i,t] * (Y[i,k] + 1) for i in range(n))) #Nkt = 1/2(sum of (1+Yik)*zit #may need to be corrected

    m_h.addConstr(N[t] == z.sum("*",t))  # Nt = sum of zit
    m_h.addConstr(l[t] == ck.sum("*",t)) # sum of ckt = lt
    m_h.addConstr(l[t] == 1, name="dt_constraint_l(t)")
 
for t in TL:
    l_ancestors = left_ancestors[t - 1]  # cache the list
    if l_ancestors:
        for la in l_ancestors:
            for i in range(n):
                xi = X.iloc[i]  # cache row once
                m_h.addConstr(gp.quicksum(a[j,la]*xi[j] for j in range(p))+mu <= b[la]+(2+mu)*(1-z[i, t]),
                    name=f"split_l_{la}_{i}_{t}"
                )
                
    r_ancestors = right_ancestors[t - 1]
    if r_ancestors:
        for ra in r_ancestors:
            for i in range(n):
                xi = X.iloc[i]  # cache the row once
                m_h.addConstr(
                    gp.quicksum(a[j, ra]*xi[j] for j in range(p)) >= b[ra]-2*(1-z[i, t]),
                    name=f"split_r_{ra}_{i}_{t}"
                )
         
for t in TL:
    m_h.addConstr(L[t] >= 0, name="Lt_constraint3") #Lt ≥ 0
    for k in range(K):
        m_h.addConstr(L[t] >= N[t] - Nk[k,t] - n*(1-ck[k,t]), name="Lt_constraint1") #Lt ≤ Nt − Nkt + n(1-ckt) 
        m_h.addConstr(L[t] <= N[t] - Nk[k,t] + n*ck[k,t], name="Lt_constraint2")  #Lt ≤ Nt − Nkt + n*ckt 

Objective OCT-H and Solution

In [33]:
m_h.update()
m_h.setObjective(L.sum('*') / L_hat + alpha*gp.quicksum(gp.quicksum(s[i,t] for i in range(p)) for t in TB), GRB.MINIMIZE) #minimize m_h.setObjective(gp.quicksum(L[t]/L_hat for t in TL) + alpha*gp.quicksum(gp.quicksum(s[i,t] for i in range(p)) for t in TB), GRB.MINIMIZE) L + alpha*sum(sum(sjt)) for j in p, t in TB
m.Params.timelimit = 600
m_h.optimize()

Set parameter TimeLimit to value 600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 9 185H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 96382 rows, 24217 columns and 768745 nonzeros
Model fingerprint: 0xff54e03e
Variable types: 64 continuous, 24153 integer (24129 binary)
Coefficient statistics:
  Matrix range     [2e-02, 3e+03]
  Objective range  [5e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]

User MIP start did not produce a new incumbent solution

Presolve removed 24269 rows and 80 columns
Presolve time: 1.06s
Presolved: 72113 rows, 24137 columns, 672180 nonzeros
Variable types: 7 continuous, 24130 integer (24106 binary)

Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.10s

Barrier statistics